<a href="https://colab.research.google.com/github/osmarbraz/exemplos_BERT/blob/main/ExemplosQuestionAnweringBERT_pt_br.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exemplo de Question Answering em português

Usando biblioteca Transformer e MCL BERT ajustado para QA.

# 1 Instalação dos pacotes das biblioteca Transformer

In [ ]:
# Instala a última versão da biblioteca
!pip install transformers

# Instala uma versão Especifica da biblioteca
# !pip install -U transformers==4.5.1

     |████████████████████████████████| 2.6 MB 3.3 MB/s 
     |████████████████████████████████| 895 kB 20.8 MB/s 
     |████████████████████████████████| 636 kB 27.8 MB/s 
     |████████████████████████████████| 3.3 MB 27.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# 2 Modelos BERT de QA Disponíveis

https://huggingface.co/models?filter=pt&pipeline_tag=question-answering


* 'pucpr/bioBERTpt-squad-v1.1-portuguese'
* 'pierreguillou/bert-base-cased-squad-v1.1-portuguese'
* 'pierreguillou/bert-large-cased-squad-v1.1-portuguese'
* 'mrm8488/bert-base-portuguese-cased-finetuned-squad-v1-pt'






# 3 Usando Classes do Huggingface



## Carrega o modelo

In [ ]:
# Importando as bibliotecas
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

nomeModeloBERT = 'mrm8488/bert-base-portuguese-cased-finetuned-squad-v1-pt'
#nomeModeloBERT = 'pierreguillou/bert-base-cased-squad-v1.1-portuguese'
#nomeModeloBERT = 'pierreguillou/bert-large-cased-squad-v1.1-portuguese'

# Carrega o tokenizador
tokenizer = AutoTokenizer.from_pretrained(nomeModeloBERT) 

# Carrega o modelo
model = AutoModelForQuestionAnswering.from_pretrained(nomeModeloBERT)

## Exemplo 1

Existem três etapas para o QA:
1. tokenizar a entrada
2. obter pontuações do modelo
3. obter a resposta

Essas etapas são discutidas em detalhes no HF [Transformer Notebooks](https://huggingface.co/transformers/notebooks.html).


In [ ]:
contexto = r"""
          A pandemia de COVID-19, também conhecida como pandemia de coronavírus, é uma pandemia em curso de COVID-19, 
          uma doença respiratória aguda causada pelo coronavírus da síndrome respiratória aguda grave 2 (SARS-CoV-2). 
          A doença foi identificada pela primeira vez em Wuhan, na província de Hubei, República Popular da China, 
          em 1 de dezembro de 2019, mas o primeiro caso foi reportado em 31 de dezembro do mesmo ano. 
          Acredita-se que o vírus tenha uma origem zoonótica, porque os primeiros casos confirmados 
          tinham principalmente ligações ao Mercado Atacadista de Frutos do Mar de Huanan, que também vendia animais vivos. 
          Em 11 de março de 2020, a Organização Mundial da Saúde declarou o surto uma pandemia. Até 8 de fevereiro de 2021, 
          pelo menos 105 743 102 casos da doença foram confirmados em pelo menos 191 países e territórios, 
          com cerca de 2 308 943 mortes e 58 851 440 pessoas curadas.
          """

pergunta = "Quando começou a pandemia de Covid-19 no mundo?"

# 1. Tokenizando a entrada e o contexto
entrada = tokenizer.encode_plus(pergunta, contexto, return_tensors="pt") 

# 2. Obtendo a pontuação do modelo
pontuacao_inicio_resposta, pontuacao_fim_resposta = model(**entrada, return_dict=False)
inicio_resposta = torch.argmax(pontuacao_inicio_resposta) 
fim_resposta = torch.argmax(pontuacao_fim_resposta) + 1 

# 3. Obtendo a resposta
resposta = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(entrada["input_ids"][0][inicio_resposta:fim_resposta]))

print(f"Resposta:", resposta)

Resposta: 1 de dezembro de 2019


## Exemplo 2

In [ ]:
contexto = r"""No dia 22 de abril comemoramos o descobrimento do Brasil. Na mesma data, 
            só que no ano de 1.500, Pedro Álvares Cabral chegava às terras de "Vera Cruz", 
            que mais tarde seriam chamadas de Brasil. 
            Ele chegou próximo à região de Porto Seguro, no estado da Bahia.
            Pedro Álvares Cabral não foi o primeiro português a pisar 
            em solo brasileiro. Na verdade, quem fez isso foi Duarte Pacheco Pereira, um navegador militar. 
            Em 1498 ele foi designado por D. Manuel I para uma expedição secreta para reconhecer as zonas situadas além da marcação de Tordesilhas. 
            A expedição dele partiu do arquipélago de Cabo Verde e chegou em algum ponto da costa entre o Maranhão e o Pará. A partir desse ponto, 
            Pereira seguido pela costa Norte, até a foz do Rio Amazonas e a Ilha de Marajó."""

pergunta = "Quem descobriu o Brasil?"

# 1. Tokenizando a entrada e o contexto
entrada = tokenizer.encode_plus(pergunta, contexto, return_tensors="pt") 

# 2. Obtendo a pontuação do modelo
pontuacao_inicio_resposta, pontuacao_fim_resposta = model(**entrada, return_dict=False)
inicio_resposta = torch.argmax(pontuacao_inicio_resposta) 
fim_resposta = torch.argmax(pontuacao_fim_resposta) + 1 

# 3. Obtendo a resposta
resposta = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(entrada["input_ids"][0][inicio_resposta:fim_resposta]))

print(f"Resposta:", resposta)

Resposta: pedro alvares cabral


# 4 Usando Pipeline do Huggingface

## Carrega o pipeline

In [ ]:
# Importando as bibliotecas
import transformers
from transformers import pipeline

In [ ]:
# Carrega o pipeline
model_qa = pipeline('question-answering', model='mrm8488/bert-base-portuguese-cased-finetuned-squad-v1-pt', tokenizer='mrm8488/bert-base-portuguese-cased-finetuned-squad-v1-pt')

## Exemplo 1

In [ ]:
contexto = r"""
          A pandemia de COVID-19, também conhecida como pandemia de coronavírus, é uma pandemia em curso de COVID-19, 
          uma doença respiratória aguda causada pelo coronavírus da síndrome respiratória aguda grave 2 (SARS-CoV-2). 
          A doença foi identificada pela primeira vez em Wuhan, na província de Hubei, República Popular da China, 
          em 1 de dezembro de 2019, mas o primeiro caso foi reportado em 31 de dezembro do mesmo ano. 
          Acredita-se que o vírus tenha uma origem zoonótica, porque os primeiros casos confirmados 
          tinham principalmente ligações ao Mercado Atacadista de Frutos do Mar de Huanan, que também vendia animais vivos. 
          Em 11 de março de 2020, a Organização Mundial da Saúde declarou o surto uma pandemia. Até 8 de fevereiro de 2021, 
          pelo menos 105 743 102 casos da doença foram confirmados em pelo menos 191 países e territórios, 
          com cerca de 2 308 943 mortes e 58 851 440 pessoas curadas.
          """

pergunta = "Quando começou a pandemia de Covid-19 no mundo?"

resultado = model_qa(question=pergunta, context=contexto)

print(" Resposta QA: ", resultado['answer'], ", pontuação: ",  round(resultado['score'], 4),  ", início: ", resultado['start'],  ", fim: ", resultado['end'])

 Resposta QA:  1 de dezembro de 2019 , pontuação:  0.6949 , início:  368 , fim:  389


## Exemplo 2

In [ ]:
pergunta = "Quem descobriu o Brasil?"

contexto = """No dia 22 de abril comemoramos o descobrimento do Brasil. Na mesma data, 
            só que no ano de 1.500, Pedro Álvares Cabral chegava às terras de "Vera Cruz", 
            que mais tarde seriam chamadas de Brasil. 
            Ele chegou próximo à região de Porto Seguro, no estado da Bahia.
            Pedro Álvares Cabral não foi o primeiro português a pisar 
            em solo brasileiro. Na verdade, quem fez isso foi Duarte Pacheco Pereira, um navegador militar. 
            Em 1498 ele foi designado por D. Manuel I para uma expedição secreta para reconhecer as zonas situadas além da marcação de Tordesilhas. 
            A expedição dele partiu do arquipélago de Cabo Verde e chegou em algum ponto da costa entre o Maranhão e o Pará. A partir desse ponto, 
            Pereira seguido pela costa Norte, até a foz do Rio Amazonas e a Ilha de Marajó."""

resultado = model_qa(question=pergunta, context=contexto)

print(" Resposta QA: ", resultado['answer'], ", pontuação: ",  round(resultado['score'], 4),  ", início: ", resultado['start'],  ", fim: ", resultado['end'])

 Resposta QA:  Pedro Álvares Cabral , pontuação:  0.9999 , início:  110 , fim:  130


# Respondendo várias perguntas de um conjunto de dados

## Instalando a biblioteca Dataset

https://huggingface.co/docs/datasets/installation.html

In [ ]:
!pip install datasets

     |████████████████████████████████| 264 kB 4.9 MB/s 
     |████████████████████████████████| 243 kB 53.0 MB/s 
     |████████████████████████████████| 118 kB 69.7 MB/s 
     |████████████████████████████████| 76 kB 3.9 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


## Carregando um conjunto de dados

Carregando o SQUAD 1.0 traduzido.

Lista dos conjunto de dados em português.
https://huggingface.co/datasets?filter=languages:pt

In [ ]:
# Import da biblioteca
from datasets import load_dataset

# Carrega o conjunto de dados
datasets = load_dataset('squad_v1_pt')

print(datasets)

Using custom data configuration default


Dataset squad_v1_pt downloaded and prepared to /root/.cache/huggingface/datasets/squad_v1_pt/default/1.1.0/65162e0fbe44f19a4d2ad9f5f507d2e965e74249fc3239dc78b4e3bd93bab7c4. Subsequent calls will reuse this data.
DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})


## Respondendo um conjunto de dados de perguntas

In [ ]:
# Importando as bibliotecas
import transformers
from transformers import pipeline

In [ ]:
# Carrega o pipeline
model_qa = pipeline('question-answering', model='mrm8488/bert-base-portuguese-cased-finetuned-squad-v1-pt', tokenizer='mrm8488/bert-base-portuguese-cased-finetuned-squad-v1-pt')

In [ ]:
# Percorre o conjunto de dados
for i, pergunta in enumerate(datasets['validation']):
  # Responde somente 10 perguntas
  if i < 10:
    print("\nPergunta:", pergunta['question'])
    print("Contexto:",pergunta['context'])
    print("       Respostas esperada:",pergunta['answers'])
    for j, resposta in enumerate(pergunta['answers']['text']):
        print("       Resposta esperada:",resposta)

    resultado = model_qa(question=pergunta['question'], context=pergunta['context'])

    print("      Resposta QA: ", resultado['answer'], ", pontuação: ",  round(resultado['score'], 4),  ", início: ", resultado['start'],  ", fim: ", resultado['end'])



Pergunta: Qual time da NFL representou o AFC no Super Bowl 50?
Contexto: Super Bowl 50 foi um jogo de futebol americano para determinar o campeão da National Football League (NFL) para a temporada de 2015. O campeão da American Football Conference (AFC), Denver Broncos, derrotou a campeã Carolina Panthers, da National Football Conference (NFC), por 24 a 10, e conquistou seu terceiro título no Super Bowl. O jogo foi disputado em 7 de fevereiro de 2016, no Levi&#39;s Stadium, na área da baía de San Francisco, em Santa Clara, Califórnia. Como este foi o 50º Super Bowl, a liga enfatizou o &quot;aniversário de ouro&quot; com várias iniciativas de ouro, bem como a suspensão temporária da tradição de nomear cada jogo do Super Bowl com algarismos romanos (sob os quais o jogo seria conhecido como &quot; Super Bowl L &quot;), para que o logotipo possa destacar os algarismos arábicos 50.
       Respostas esperada: {'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'], 'answer_start': [